In [2]:
import os
import sys
import pickle
from faker import Faker
f = Faker()

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)

models_list = pickle.load(open("models_new.pkl", "rb"))
models_dict = models_list[0]
print(models_dict.keys())

from datetime import datetime

from src.socotra_loader.requests import *
from src.entities import *
from src.cache_controller import *

/home/lytvyn/projects/jetty/jetty-migration
dict_keys(['PolicyHolder', 'Policies'])


In [3]:
redis_controller

### Caching scenario

1. Load policy owner
    1. Add policy locators "policy_locators"
    2. Upload Entity to "PolicyHolder" folder
    
2. Load Policy entities
    1. Create fields
        - "auxdata_locators"
        - "endorsement_locators"
        - "renewal_locators"
    2. Upload Entity to "Policy" folder
    3. Append created policy_locator to "PolicyHolder:<policyholder_locator>.policy_locators"
3. Load AuxData
    1. Create Entity
    2. Load to "AuxData" folder
    3. Append auxdata_locator to "Policy:<policy_locator>.auxdata_locators"
4. Load Endorsement(s)
    1. Create Entity
    2. Load to "Endosements" folder
    3. Append endorsement_locator to "Policy:<policy_locator>.endorsement_locators"
4. Load Renewal(s)
    1. Create Entity
    2. Load to "Renewals" folder
    3. Append renewal_locator to "Policy:<policy_locator>.renewal_locators"
     

### Policy Holder CRU

In [ ]:
ph = models_dict['PolicyHolder']
ph_locator = redis_controller.save_item(item=ph, item_status = "constructed")

# redis_controller.get_item(entity_type="PolicyHolder", locator=ph_locator, item_status = "constructed")
# r_controller.set_entity_uploaded(entity_type="PolicyHolder", locator="AA1111")
# r_controller.get_item(entity_type="PolicyHolder", locator="AA1111", item_status = "constructed")

### Policy CRU

In [ ]:
policy = models_dict['PolicyProtect']
policy_locator = redis_controller.save_item(item=policy, item_status = "constructed", corresponding_locator=ph_locator)

assert isinstance(redis_controller.get_item(entity_type=policy.name, locator=policy_locator, item_status = "constructed"), Policy)
redis_controller.set_entity_uploaded(entity_type=policy.name, locator=policy.id)
assert isinstance(redis_controller.get_item(entity_type=policy.name, locator=policy_locator, item_status = "constructed"), Policy)

### Aux Data CRU

In [ ]:
aux_data = models_dict['AuxData']
auxdata_locator = redis_controller.save_item(item=aux_data, item_status="constructed", corresponding_locator=policy_locator)
assert isinstance(redis_controller.get_item(entity_type=aux_data.name, locator=auxdata_locator, item_status = "constructed"), AuxData)
redis_controller.set_entity_uploaded(entity_type="AuxData", locator=auxdata_locator)
assert isinstance(redis_controller.get_item(entity_type=aux_data.name, locator=auxdata_locator, item_status = "constructed"), AuxData)

### Endorsement CRU

In [ ]:
endorsement = models_dict['Endorsment']
endorsement_locator = redis_controller.save_item(item=endorsement, item_status = "constructed", corresponding_locator=policy_locator)
assert isinstance(redis_controller.get_item(entity_type=endorsement.name, locator=endorsement_locator, item_status = "constructed"), Endorsement)
redis_controller.set_entity_uploaded(entity_type=endorsement.name, locator=endorsement_locator)
assert isinstance(redis_controller.get_item(entity_type=endorsement.name, locator=endorsement_locator, item_status = "constructed"), Endorsement)

### Renewal CRU

In [ ]:
renewal = models_dict['Renewal']
renewal_locator = redis_controller.save_item(item=renewal, item_status = "constructed", corresponding_locator=policy_locator)
assert isinstance(redis_controller.get_item(entity_type=renewal.name, item_status="constructed", locator=renewal_locator), Renewal)
redis_controller.set_entity_uploaded(entity_type=renewal.name, locator=renewal_locator)
assert isinstance(redis_controller.get_item(entity_type="Renewal", item_status="constructed", locator=renewal_locator), Renewal)

In [5]:
STATUS = "constructed"

for ph_entry in models_list:
    ph_locator = redis_controller.save_item(item=ph_entry["PolicyHolder"], item_status=STATUS)
    # logging "PolicyHolder" <locator> has been saved
    if ph_entry["Policies"]:
        policy_locator = None
        for policy_data in ph_entry["Policies"]:
            for key, item in policy_data.items():
                if key == "Policy":
                    result = redis_controller.save_item(item=item, item_status=STATUS, corresponding_locator=ph_locator.value)
                    if result.ok():
                        policy_locator = result.value 
                else:
                    result = redis_controller.save_item(item=item, item_status=STATUS, corresponding_locator=policy_locator)
                    
            
    

2023-07-05 14:30:26.895 | INFO     | src.cache_controller:save_item:117 - Saving constructed:PolicyHolder with key AA1111
2023-07-05 14:30:26.896 | INFO     | src.cache_controller:save_item:117 - Saving constructed:PolicyHolder with key AA2222
2023-07-05 14:30:26.897 | INFO     | src.cache_controller:save_item:117 - Saving constructed:Policy with key AA4590
2023-07-05 14:30:26.898 | INFO     | src.cache_controller:save_item:117 - Saving constructed:AuxData with key c6c32a0d-39e6-4055-8b64-f7d4e9708b24
2023-07-05 14:30:26.899 | INFO     | src.cache_controller:save_item:117 - Saving constructed:Endorsement with key 663e422a-5a2e-4598-abc2-d5b3cdd4b115
2023-07-05 14:30:26.899 | INFO     | src.cache_controller:save_item:117 - Saving constructed:Renewal with key 2877fa01-3fd1-4f82-a5ec-e4f8c450293f
2023-07-05 14:30:26.900 | INFO     | src.cache_controller:save_item:117 - Saving constructed:Policy with key AA2223
2023-07-05 14:30:26.901 | INFO     | src.cache_controller:save_item:117 - Savin

In [4]:
str(redis_controller.key_manager)

"{'PolicyHolder': {'AA1111': [], 'AA2222': ['AA4590', 'AA2223'], 'AA3333': ['AA3331', 'AA3332']}, 'Policy': {'AA4590': {'AuxData': ['a9cf36c6-f21d-4acb-80fd-699af9aab369'], 'Endorsement': ['970a32e8-c742-46a5-9bff-37cd648a0f1c'], 'Renewal': ['9ac4d939-9a09-4aa4-9b66-f0c98005379b']}, 'AA2223': {'AuxData': ['b9829906-f3c1-47cc-9494-7c5e84c29678'], 'Endorsement': ['22b0935d-93c9-4e36-978a-f9692109d3b3'], 'Renewal': []}, 'AA3331': {'AuxData': ['e45c3954-77af-422e-8efa-2fd9cd9a57a8'], 'Endorsement': [], 'Renewal': []}, 'AA3332': {'AuxData': ['fe3ef74f-26ae-40f6-a3f9-9ab5d8c6eaec'], 'Endorsement': ['301baafb-3b5e-4498-a5d2-0ecba4032260'], 'Renewal': ['5e6a92fb-f32f-4e95-ac7b-0d32903e4ed1']}}}"

In [5]:
models_list[2]["Policies"][0]["Policy"].id

'AA3331'

In [6]:
ph_policy = redis_controller.key_manager.KEYS["PolicyHolder"]
policy_main = redis_controller.key_manager.KEYS["Policy"]

for ph_key, policy_keys in ph_policy.items():
    # retrieve PH
    # create PH
    if not policy_keys:
        # logg "no policies for policyholder <locator>"
        continue
    else:
        # logg 
        for policy_key in policy_keys:
            print(policy_key)
            # create context
            print(
                redis_controller.get_item(
                    entity_type="Policy",
                    locator=policy_key,
                    item_status=STATUS
                )
            )
            for item_name, locators  in policy_main[policy_key].items():
                for locator in locators:
                    print(
                        redis_controller.get_item(
                            entity_type=item_name,
                            locator=locator,
                            item_status=STATUS
                        ))
    

AA4590
Ok(Policy(retrieved_from_brighcore=None, uploaded_to_socotra=None, policyholderLocator='8b045c1c-28ce-416e-944c-473706fbff42', productName='Protect', policyStartTimestamp=datetime.datetime(2023, 6, 23, 9, 15, 47), policyEndTimestamp=datetime.datetime(2024, 6, 25, 23, 16, 17), exposures=Exposure(exposureName='exposure', fieldValues={'iso_protection_class': 'Try to transmit the SDD port, maybe it will program the cross-platform panel!'}, fieldGroups=None, perils=[Peril(name='contents', deductible=200), Peril(name='loss_of_use', deductible=500)]), fieldValues=PolicyFieldValues(rater_source='AA2221', rater_id='AA4590', credit_score=None, custom_policy_number='AA4590', protection_class=None, renewal_term=0, auto_renewal_status='renew', policy_purchase_timestamp=datetime.datetime(2023, 6, 23, 7, 15, 47, tzinfo=datetime.timezone.utc), number_of_people_on_policy=1, all_peril_deductible=100, has_restricted_dog_breed='no', additional_insured=0, superior_construction='None', protective_dev

In [ ]:
redis_controller.key_manager.KEYS["Policy"]